In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
!pip install ivis

     |████████████████████████████████| 655kB 5.0MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391581 sha256=6d766cf26109d47c6587a6d98776838d5504ddc289771ee70bba2c299ffe6af8
  Stored in directory: /root/.cache/pip/wheels/3a/c5/59/cce7e67b52c8e987389e53f917b6bb2a9d904a03246fadcb1e
Successfully built annoy


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.metrics.scorer import make_scorer
from sklearn.metrics import confusion_matrix, average_precision_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.utils import shuffle
from sklearn import preprocessing
import seaborn as sns
from sklearn.metrics import roc_auc_score, auc

import time
import warnings
warnings.filterwarnings('ignore')

In [6]:
train_df = pd.read_csv("/gdrive/My Drive/Colab_Notebooks/Sentence-Analysis/input.csv")
test_df =pd.read_csv("/gdrive/My Drive/Colab_Notebooks/Sentence-Analysis/input_test.csv")

mappings = {1: 0, 2: 1, 3:2}

train_df.label = [mappings[item] for item in train_df.label]
train_df.head()

,Unnamed: 0,patient_id,sequence,label
0,0,Z8399761,"(unclear history), venous insufficiency, PVD, ...",2
1,1,Z11957588,period. It apparently cuts car accident rates ...,0
2,2,Z10809276,(P) 5 Hearing: no concerns Vision: needs updat...,1
3,3,Z8539944,"remission, s/p 1 suicide attempt, 3 IPLOC admi...",2
4,4,Z12696803,should be pursued at that point in time. He an...,0


In [7]:
X = train_df["sequence"]
y = train_df["label"]

y_label = y.to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0,test_size=0.3, stratify = y_label)

In [8]:
# Initialize a CountVectorizer object: count_vectorizer
count_vectorizer = CountVectorizer(stop_words="english",analyzer='word')

# Transform the training data using only the 'text' column values: count_train 
count_train = count_vectorizer.fit_transform(X_train)
count_test = count_vectorizer.transform(X_test)
#count_valid = count_vectorizer.transform(X_valid)

# Create the CountVectorizer DataFrame: count_train
count_train = pd.DataFrame(count_train.A, columns=count_vectorizer.get_feature_names())

# Create the CountVectorizer DataFrame: test_df
count_test = pd.DataFrame(count_test.A, columns=count_vectorizer.get_feature_names())

#count_valid = pd.DataFrame(count_valid.A, columns=count_vectorizer.get_feature_names())
count_train.head()

,10,11,15,16,18,2017,22,38611851,50,70,abscess,accompanied,administered,admissions,age,alexander,alzheimer,ambulatory,amyloid,angiopathy,anna,annual,anticoagulation,antiplatelets,anxiety,appendectomy,assessment,assistance,attempt,awareness,behavioral,benign,blake,block,branch,bso,bundle,bwh,cataract,cerebral,...,progressive,provider,pursued,pvd,recent,referral,referred,referring,regarding,related,relation,remission,renal,report,results,review,rituxan,seizure,significant,status,stenting,suicide,summary,systems,tah,thigh,time,tracheomalacia,transient,ulcerations,unclear,upset,venous,verbal,vertigo,visit,volume,weight,wife,years
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,2,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
tfidf_vectorizer = TfidfVectorizer(stop_words="english",analyzer='word')
tfidf_train= tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)

# Create the TfidfVectorizer DataFrame: tfidf_df
tfidf_train  = pd.DataFrame(tfidf_train.A, columns=tfidf_vectorizer.get_feature_names())
tfidf_test  = pd.DataFrame(tfidf_test.A, columns=tfidf_vectorizer.get_feature_names())

tfidf_train.head()

,10,11,15,16,18,2017,22,38611851,50,70,abscess,accompanied,administered,admissions,age,alexander,alzheimer,ambulatory,amyloid,angiopathy,anna,annual,anticoagulation,antiplatelets,anxiety,appendectomy,assessment,assistance,attempt,awareness,behavioral,benign,blake,block,branch,bso,bundle,bwh,cataract,cerebral,...,progressive,provider,pursued,pvd,recent,referral,referred,referring,regarding,related,relation,remission,renal,report,results,review,rituxan,seizure,significant,status,stenting,suicide,summary,systems,tah,thigh,time,tracheomalacia,transient,ulcerations,unclear,upset,venous,verbal,vertigo,visit,volume,weight,wife,years
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.263644,0.000000,0.0,0.0,0.0,0.000000,0.263644,0.0,0.0,0.0,0.263644,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.228188,0.0,0.0,0.0,0.0,0.263644,0.00000,0.0,0.263644,0.0,0.00000,0.00000,0.0,0.00000,0.0,0.263644,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.00000,0.0,0.000000,0.313426,0.000000,0.0,0.0,0.0,0.0,0.0,0.313426,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.00000,0.00000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.23107,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.23107,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.23107,0.0,0.000000,0.0,0.23107,0.23107,0.0,0.23107,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.203012,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.156338,0.203012,0.0,0.0,0.406024,0.0,0.0,0.0,0.0,0.000000,0.203012,0.0,0.0,0.0,0.203012,0.000000,0.0,0.0,0.0,0.000000,0.0,0.203012,0.0,0.0,...,0.0,0.203012,0.0,0.00000,0.0,0.203012,0.000000,0.203012,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.00000,0.00000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.191994,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.00000,0.00000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [12]:
def filter_features_by_cor(df):
    m = len(df.columns)
    output = df.iloc[:,m-1] 
    output_list = output.tolist()
    corrcoef_array = []

    for i in range(0,m-2):
        input_list = df.iloc[:,i].tolist()
        pair_v = [input_list, output_list]
        #print(pair_v)
        corrcoef = abs(np.corrcoef(pair_v)) #each row should represent a variable
        corrcoef_array = np.append(corrcoef_array,corrcoef[0,1])

    feature_names = list(df)
    feature_names = feature_names[0:m-2]
    output_df = pd.DataFrame(feature_names, columns=['Features'])
    output_df['CorrCoef'] = corrcoef_array
    output_df = output_df.sort_values('CorrCoef')
    output_df = output_df.reset_index()
    output_df = output_df.drop(columns = "index")
    
    return output_df

In [14]:
tfidf_features_df = pd.concat([tfidf_train, y_train.reset_index(drop=True)], axis=1)
tfidf_features_df.head()

,10,11,15,16,18,2017,22,38611851,50,70,abscess,accompanied,administered,admissions,age,alexander,alzheimer,ambulatory,amyloid,angiopathy,anna,annual,anticoagulation,antiplatelets,anxiety,appendectomy,assessment,assistance,attempt,awareness,behavioral,benign,blake,block,branch,bso,bundle,bwh,cataract,cerebral,...,provider,pursued,pvd,recent,referral,referred,referring,regarding,related,relation,remission,renal,report,results,review,rituxan,seizure,significant,status,stenting,suicide,summary,systems,tah,thigh,time,tracheomalacia,transient,ulcerations,unclear,upset,venous,verbal,vertigo,visit,volume,weight,wife,years,label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.263644,0.000000,0.0,0.0,0.0,0.000000,0.263644,0.0,0.0,0.0,0.263644,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.228188,0.0,0.0,0.0,0.0,0.263644,0.00000,0.0,0.263644,0.0,0.00000,0.00000,0.0,0.00000,0.0,0.263644,0.0,0.0,0.0,0.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.313426,0.000000,0.0,0.0,0.0,0.0,0.0,0.313426,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.00000,0.00000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,2
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.23107,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.23107,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.23107,0.0,0.000000,0.0,0.23107,0.23107,0.0,0.23107,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,2
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.203012,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.156338,0.203012,0.0,0.0,0.406024,0.0,0.0,0.0,0.0,0.000000,0.203012,0.0,0.0,0.0,0.203012,0.000000,0.0,0.0,0.0,0.000000,0.0,0.203012,0.0,0.0,...,0.203012,0.0,0.00000,0.0,0.203012,0.000000,0.203012,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.00000,0.00000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,2
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.191994,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.00000,0.00000,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0


In [15]:
tfidf_output_df = filter_features_by_cor(tfidf_features_df)
tfidf_output_df = tfidf_output_df.sort_values(by=['CorrCoef'],ascending = False)
tfidf_output_df.to_csv('tfidf_vector_feature_corr.csv')

tfidf_output_df

,Features,CorrCoef
194,heart,0.563488
189,point,0.543928
184,pursued,0.543928
185,following,0.543928
187,prior,0.543928
...,...,...
4,memory,0.053246
3,dementia,0.050787
2,patient,0.044448
1,changes,0.021526


In [16]:
lr_df_list  = []
svm_df_list = []

lr_mcc_list = []
lr_auc_list = []
lr_c_list   = []

svm_acc_list = []
svm_mcc_list = []
svm_auc_list = []

# Setting Correlation threshold
top_tfidf_features_df = tfidf_output_df[:500]
filtered_tfidf_train = tfidf_train.filter(items=top_tfidf_features_df['Features'])
filtered_tfidf_test = tfidf_test.filter(items=top_tfidf_features_df['Features'])

# svm = LinearSVC(probability=True)
svm = SVC(probability=True, kernel='linear')
svm.fit(filtered_tfidf_train, y_train)
y_prob_svm = svm.predict_proba(filtered_tfidf_test)[:,1]
labels_svm = svm.predict(filtered_tfidf_test)

#svm_auc_list.append(metrics.roc_auc_score(y_test, y_prob_svm))
svm_auc_df = pd.DataFrame(svm_auc_list, columns=['auc'])
# svm_auc_df['corr_thres'] = corr
svm_df_list.append(svm_auc_df)

In [17]:
target_names = ['NO', 'NTR', 'YES']
results_svm = classification_report(y_test, labels_svm, target_names=target_names, output_dict=True)
results_svm = pd.DataFrame(results_svm).transpose()
print(results_svm)
print("\nConfusion Matrix: \n", confusion_matrix(y_test, labels_svm))

              precision    recall  f1-score   support
NO             0.000000  0.000000  0.000000  1.000000
NTR            0.000000  0.000000  0.000000  2.000000
YES            0.571429  1.000000  0.727273  4.000000
accuracy       0.571429  0.571429  0.571429  0.571429
macro avg      0.190476  0.333333  0.242424  7.000000
weighted avg   0.326531  0.571429  0.415584  7.000000

Confusion Matrix: 
 [[0 0 1]
 [0 0 2]
 [0 0 4]]


In [18]:
labels_lgr_tfidf = np.array([])

for c in [0.01, 0.1, 1, 10, 100]:
    lr = LogisticRegression(C=c, random_state=0)
    lr.fit(filtered_tfidf_train, y_train)
    labels_lgr_tfidf = lr.predict(filtered_tfidf_test)

    results_lgr = classification_report(y_test, labels_lgr_tfidf, target_names=target_names, output_dict=True)
    results_lgr = pd.DataFrame(results_lgr).transpose()
    print("\nC: ", c, "\n", results_svm)
    print("\nConfusion Matrix: \n", confusion_matrix(y_test, labels_lgr_tfidf))

    print("---------------------------------------------------------------------\n")

    y_prob_lr = lr.predict_proba(filtered_tfidf_test)[:,1]
    #lr_auc_list.append(metrics.roc_auc_score(y_test, y_prob_lr))
    lr_c_list.append(c)
    
lr_auc_df = pd.DataFrame(lr_auc_list, columns=['auc'])
lr_c_df = pd.DataFrame(lr_c_list, columns=['c_value'])
lr_iter_df = pd.concat([lr_c_df, lr_auc_df], axis=1)
# lr_iter_df['corr_thres'] = corr
lr_df_list.append(lr_iter_df)


C:  0.01 
               precision    recall  f1-score   support
NO             0.000000  0.000000  0.000000  1.000000
NTR            0.000000  0.000000  0.000000  2.000000
YES            0.571429  1.000000  0.727273  4.000000
accuracy       0.571429  0.571429  0.571429  0.571429
macro avg      0.190476  0.333333  0.242424  7.000000
weighted avg   0.326531  0.571429  0.415584  7.000000

Confusion Matrix: 
 [[0 0 1]
 [0 0 2]
 [0 0 4]]
---------------------------------------------------------------------


C:  0.1 
               precision    recall  f1-score   support
NO             0.000000  0.000000  0.000000  1.000000
NTR            0.000000  0.000000  0.000000  2.000000
YES            0.571429  1.000000  0.727273  4.000000
accuracy       0.571429  0.571429  0.571429  0.571429
macro avg      0.190476  0.333333  0.242424  7.000000
weighted avg   0.326531  0.571429  0.415584  7.000000

Confusion Matrix: 
 [[0 0 1]
 [0 0 2]
 [0 0 4]]
----------------------------------------------------

In [ ]:
# re run after issue with calculating metrics for datasets with 3 classes is resolved
# lr_df = pd.concat(lr_df_list, axis=0)
# svm_df = pd.concat(svm_df_list, axis=0)
# print(lr_df.head())
# print('\n')
# print(svm_df.head())